In [1]:
# encoding=utf-8
# 浪潮之巅　实体分析

import jieba
import codecs
with codecs.open('clzd.txt','r','utf-8') as f:
    all_words = f.readlines()
    lines = []
    cnt = 0
    for line in all_words:
        if line != "\r\n":
            line = line[0:-2]
            lines.append(line)
            cnt += len(line)
    #print(lines)
    print(cnt)

175881


In [2]:
import collections
word_cnt = collections.defaultdict(int)
words_list = []
for line in lines:
    li = jieba.cut(line)
    for item in li:
        word_cnt[item] += 1
#print(word_cnt)
print(len(word_cnt))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.665 seconds.
Prefix dict has been built succesfully.


9972


In [3]:
stopwords = set()
# 载入停用词(哈工大停用词)
with codecs.open('hit_stopwords.txt','r','utf-8') as f:
    words = f.readlines()
    for word in words:
        stopwords.add(word[:-2])
        
word_list = [(k,v) for k,v in word_cnt.items() if k not in stopwords]
print(len(word_list))

sorted_list = sorted(word_list, key=lambda t:(-t[1]))
print(sorted_list[10:50])

9659
[('人', 294), ('年', 269), ('美国', 249), ('硅谷', 237), ('中', 234), ('后', 215), ('英特尔', 211), ('技术', 210), ('很多', 209), ('投资', 208), ('摩托罗拉', 203), ('已经', 194), ('发展', 193), ('计算机', 192), ('做', 183), ('会', 182), ('世界', 182), ('惠普', 179), ('大', 175), ('处理器', 174), ('.', 170), ('可能', 163), ('时', 161), ('思科', 156), ('太阳', 155), ('AT&T', 155), ('互联网', 154), ('今天', 149), ('还', 149), ('苹果', 148), ('产品', 147), ('新', 147), ('不是', 142), ('成功', 142), ('手机', 138), ('现在', 137), ('领域', 137), ('%', 136), ('微机', 136), ('最', 135)]


In [4]:
import os
LTP_DIR_PATH = '/home/alexsun/ML/nlp/ltp_model/ltp_data'
pos_model_path = os.path.join(LTP_DIR_PATH, 'pos.model')
# 词性标注模型路径，模型名称为`pos.model`

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型
post_lines = []
# 分词后的列表，每项代表一段分词结果
paragh_words = []
for line in lines:
    # 不考虑太短的句子
    if len(line) < 20:
        continue
    li = list(jieba.cut(line))
    new_li = []
    for item in li:
        if item not in stopwords:
            new_li.append(item)
    paragh_words.append(new_li)


In [5]:
# 每项为一段文字的词性标签
all_paragh_tags = []
# 每项为一段文字的分词结果
paragh_words_list = paragh_words
for line in paragh_words_list:
    line_tages = postagger.postag(line)
    all_paragh_tags.append(list(line_tages))
print(all_paragh_tags[13])
print(paragh_words_list[13])
print(len(all_paragh_tags))

['v', 'v', 'v', 'v', 'nt', 'v', 'nz', 'n', 'n', 'i', 'z', 'nt', 'd', 'v', 'i', 'ws', 'a', 'v', 'v', 'm', 'd', 'v', 'n', 'i', 'd', 'nt', 'n', 'v', 'nt', 'ws', 'nt', 'a', 'nh', 'n', 'nl', 'nh', 'n', 'm', 'n', 'd', 'v', 'v', 'a', 'n', 'n', 'n', 'n', 'n', 'n', 'ns', 'd', 'v', 'ws', 'n', 'v', 'n', 'v', 'd', 'nt', 'r', 'n', 'ws', 'd', 'm', 'a', 'v', 'n', 'm', 'q', 'n', 'v', 'nt', 'nd', 'v', 'd', 'v', 'v', 'ni', 'd', 'v', 'a', 'n', 'd', 'n', 'v', 'nd', 'v', 'n']
[' ', ' ', ' ', ' ', '一九九五年', '说', 'AT&T', '公司', '顶峰', '接下来', '短短的', '十年', '便', '分崩离析', '不复存在', 'AT&T', '不紧不慢', '向上', '走过', '百年', '才', '爬', '顶点', '走下坡路', '却', '十年', '时间', '注', '今天', 'AT&T', '当年', '小', '贝尔', '公司', '西南', '贝尔', '公司', '几次', '小吃', '大', '合并', '出', '类似', '水电', '公司', '设施', '服务公司', '类', '公司', '美国', '统统', '称为', 'utility', '公司', '毫无', '技术', '可言', '其实', '一九九五年', '这十来', '年间', 'AT&T', '本来', '两次', '绝佳', '发展', '机遇', '2000', '年', '网络', '革命', '九十年代', '中期', '延续', '至今', '无线通信', '飞跃', 'AT&T', '没有', '利用', '好', '机会', '反而', '两场', '变革', '中', 

In [6]:
#　命名实体识别

# 命名实体识别模型路径，模型名称为`pos.model`
ner_model_path = os.path.join(LTP_DIR_PATH, 'ner.model')
from pyltp import NamedEntityRecognizer
recognizer = NamedEntityRecognizer() #模型初始化
recognizer.load(ner_model_path)
len_paragh = len(all_paragh_tags)
# 每一项为一段文字的实体标注结果
netags_all = []
for i in range(len_paragh):
    netags = recognizer.recognize(paragh_words_list[i],all_paragh_tags[i])
    netags_all.append(list(netags))
recognizer.release()
print(len(netags_all))

713


In [10]:
# print(netags_all[3])
# print(paragh_words_list[3])
# print(len(netags_all[3]))
# print(len(paragh_words_list[3]))
# print(netags_all[33:50])

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-Ns', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-Nh', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-Nh', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [11]:
# 构建段落人名实体集合
# 每一项是一个段落中出现的人名实体的集合
paragh_entity_name_words = []
for i in range(len_paragh):
    this_paragh_words = paragh_words_list[i]
    this_paragh_ntags = netags_all[i]
    li = []
    for i in range(len(this_paragh_ntags)):
        item = this_paragh_ntags[i]
        if item == 'S-Nh':
            li.append(this_paragh_words[i])
    paragh_entity_name_words.append(li)
print(paragh_entity_name_words)
            

[['余辉'], ['吴军', '吴军'], [], [], ['高盛'], ['弗伦'], [], [], [], [], ['梅毅强', '江泽民'], [], ['余辉'], [], [], ['道琼斯'], ['小得多'], [], [], ['余辉'], [], [], ['弗伦'], [], [], [], ['阿尔卡特'], [], [], ['余辉'], [], [], ['汤普生'], [], [], ['巴菲特', '巴菲特'], [], [], ['卡斯帕罗夫', '卡斯帕罗夫', '卡斯帕罗夫', '卡斯帕罗夫'], [], [], ['托马斯'], [], ['罗斯福'], [], ['杜鲁门'], ['老华生'], [], [], [], ['奇莱'], ['小华生', '诺伊曼'], [], [], [], [], [], [], [], ['凯利', '凯利'], [], [], [], ['老华生', '小华生', '费劲儿'], [], ['盖茨', '拿破仑', '凯撒', '盖茨'], ['盖茨', '盖茨', '盖茨', '盖茨', '盖茨'], [], [], ['郭士纳', '郭士纳', '郭士纳'], ['郭士纳', '郭士纳'], [], ['郭士纳', '郭士纳', '郭士纳', '郭士纳'], ['郭士纳', '郭士纳'], ['郭士纳', '郭士纳'], ['郭士纳', '郭士纳', '郭士纳', '郭士纳', '郭士纳'], [], [], [], [], [], ['郭士纳', '巴菲特'], [], [], [], [], [], [], [], [], [], ['乔布斯'], ['阿甘'], ['蒂诺'], [], ['乔布斯', '乔布斯'], ['乔布斯', '乔布斯', '乔布斯', '乔布斯', '乔布斯'], ['乔布斯', '乔布斯', '乔布斯', '乔布斯', '乔布斯', '乔布斯'], ['乔布斯', '韦恩', '韦恩', '乔布斯'], ['乔布斯', '乔布斯', '乔布斯', '托什'], ['托什'], ['托什', '帕罗'], ['乔布斯', '乔布斯'], ['乔布斯'], ['乔布斯', '库利', '库利', '乔布斯', '库利', '乔布斯'], ['库利'

In [10]:
paragh_entity_word_set = []
for i in range(len(paragh_entity_words)):
    paragh_entity_word_set.append(set(paragh_entity_words[i]))

In [11]:
import pickle
with open('entity_set.pickle','w') as f:
    pickle.dump(paragh_entity_word_set,f)
    print("success!")

TypeError: write() argument must be str, not bytes